# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(uk_food_db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
results = establishments.count_documents(query)
print(f"---------------------------------------------------------------------------------------------------------")
print(f"There are {results} establishments with a 'Hygiene' score of 20.")
print(f"---------------------------------------------------------------------------------------------------------")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)
print(f"---------------------------------------------------------------------------------------------------------")


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f"---------------------------------------------------------------------------------------------------------")
count_of_hygiene_df = len(hygiene_df)
print(f"There are {count_of_hygiene_df} rows within this Hygiene Dataframe")
print(f"---------------------------------------------------------------------------------------------------------")
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': "London"},"RatingValue":{'$gte':"4"}}

# Use count_documents to display the number of documents in the result
results = establishments.count_documents(query)
print(f"---------------------------------------------------------------------------------------------------------")
print(f"There are {results} establishments with a 'Rating' score of 4 or above.")
print(f"---------------------------------------------------------------------------------------------------------")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)
print(f"---------------------------------------------------------------------------------------------------------")

In [ ]:
# Convert the result to a Pandas DataFrame
rating_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f"---------------------------------------------------------------------------------------------------------")
count_of_rating_df = len(rating_df)
print(f"There are {count_of_rating_df} rows within this Hygiene Dataframe")
print(f"---------------------------------------------------------------------------------------------------------")

# Display the first 10 rows of the DataFrame
rating_df.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": "5",
    'geocode.latitude': {
        '$lte': latitude + degree_search,
        '$gte': latitude - degree_search
    },
    'geocode.longitude': {
        '$lte': longitude + degree_search,
        '$gte': longitude - degree_search
    }
}

#Sort by hygiene score
sort = [('scores.Hygiene', 1)]

# Print the results
for i in establishments.find(query).sort(sort).limit(5):
    pprint(i)


In [ ]:
# Convert result to Pandas DataFrame
top5_penang_flavours_df = pd.DataFrame(establishments.find(query))
top5_penang_flavours_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Create match query
match_query = {"$match": {'scores.Hygiene': 0}}

# Grouped by "Local Authority"
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

# Then sort by classification in alphabetical order
sort_values = {'$sort': {'count': -1}}

# Create 'pipeline' function
pipeline = [match_query, group_query, sort_values]

# # Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print(f"Number of documents in result: {len(results)}")

# Print the first 10 results
for i in results[:10]:
    print(f"---------------------------------------------------------------------------------------------------------")
    print(i)


In [ ]:
# Convert the result to a Pandas DataFrame
local_authority_df = pd.DataFrame(results)
local_authority_df = local_authority_df.rename(columns={"_id": "ID", "count": "Count"})


# Display the number of rows in the DataFrame
print("----------------------------------------------------------------------------------------------")
count_of_local_authority_df = len(local_authority_df)
print(f"There are {count_of_local_authority_df} rows within this Local Authority count DataFrame")
print("----------------------------------------------------------------------------------------------")

# Display the first 10 rows of the DataFrame
print(local_authority_df.head(5))
